In [1]:
import open3d.ml as _ml3d
import open3d as o3d
import open3d.ml.torch as ml3d  # just switch to open3d.ml.tf for tf usage
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
cfg_file = "configs/kpconv_toronto3d_ADJUSTEDFORIIITB.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.KPFCNN(**cfg.model)
cfg.dataset['dataset_path'] = "/mnt/d/Krishna/Research/Datasets/Toronto3D"
dataset = ml3d.datasets.Toronto3D(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

# download the weights.
ckpt_folder = "pretrained/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "kpconv_toronto3d_202010081102utc.pth"
randlanet_url = "https://storage.googleapis.com/open3d-releases/model-zoo/kpconv_toronto3d_202010081102utc.pth"
if not os.path.exists(ckpt_path):
    cmd = "wget {} -O {}".format(randlanet_url, ckpt_path)
    os.system(cmd)

# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

test_split = dataset.get_split("test")
data = test_split.get_data(0)

'''print(test_split)
print(data)
break'''

# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)

# evaluate performance on the test set; this will write logs to './logs'.
pipeline.run_test()

preprocess: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.85s/it]

test 0/1: 100%|████████████████████████████████████████████████████████████████| 104263/104263 [02:43<00:00, 636.91it/s]

test 0/1: 100%|███████████████████████████████████████████████████████████████| 104263/104263 [00:26<00:00, 2058.22it/s]

In [5]:
len(result['predict_labels'])

2275320

In [6]:
set(result['predict_labels'])

{2, 3, 4, 5, 6, 7}

In [7]:
newData = data

In [8]:
newData['label'] = result['predict_labels']

In [9]:
newData

{'point': array([[-6.2729075e+05, -4.8419450e+06, -9.4900000e-01],
        [-6.2729075e+05, -4.8419450e+06, -9.5599997e-01],
        [-6.2729081e+05, -4.8419450e+06, -9.6100003e-01],
        ...,
        [-6.2728850e+05, -4.8419530e+06,  7.0099998e-01],
        [-6.2728850e+05, -4.8419530e+06,  7.2666663e-01],
        [-6.2728850e+05, -4.8419535e+06,  7.2500002e-01]], dtype=float32),
 'feat': array([[  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        ...,
        [147.,  49., 147.],
        [150.,  50., 150.],
        [147.,  49., 147.]], dtype=float32),
 'label': array([4, 4, 4, ..., 4, 4, 4])}

In [10]:
import numpy as np

# Assuming newData is already defined as in your example

# Combine the point cloud data: x, y, z, r, g, b, label
# Ensure colors are normalized to [0, 1] range
colors = newData['feat'][:, :3] / 255.0
labels = newData['label'].reshape(-1, 1).astype(np.float32)
data_to_write = np.hstack((newData['point'], colors, labels))

# Create a header for the PCD file
header = f"""# .PCD v0.7 - Point Cloud Data file format
VERSION 0.7
FIELDS x y z rgb label
SIZE 4 4 4 4 4
TYPE F F F F F
COUNT 1 1 1 1 1
WIDTH {data_to_write.shape[0]}
HEIGHT 1
VIEWPOINT 0 0 0 1 0 0 0
POINTS {data_to_write.shape[0]}
DATA ascii
"""

# Convert RGB values from float (0-1) to packed float (for PCD)
rgb_packed = np.array(
    ((colors[:, 0] * 255).astype(np.uint32) << 16) |
    ((colors[:, 1] * 255).astype(np.uint32) << 8) |
    ((colors[:, 2] * 255).astype(np.uint32))
).view(np.float32).reshape(-1, 1)

# Combine points, packed RGB, and labels
pcd_data = np.hstack((newData['point'], rgb_packed, labels))

# Write to the PCD file
with open("trntoIIITB.pcd", "w") as f:
    f.write(header)
    np.savetxt(f, pcd_data, fmt="%.6f %.6f %.6f %.6f %.1f")

print("Point cloud with labels saved to output_with_labels.pcd")

Point cloud with labels saved to output_with_labels.pcd
